In [1]:

import time
import torch
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
import random
from experiments.pytorch_tablu.utils import preprocess
# import numpy as np
import pandas as pd
# import wandb
from rich import print

from typing import Callable
from datetime import datetime

from time import time
from pytorch_tabular import model_sweep
import warnings

# from pytorch_tabular import TabularModel
# from pytorch_tabular.models import CategoryEmbeddingModelConfig
from pytorch_tabular.config import (
    DataConfig,
    OptimizerConfig,
    TrainerConfig,
    ExperimentConfig,
)
from pytorch_tabular.models.common.heads import LinearHeadConfig


from pytorch_tabular.utils import (
    make_mixed_dataset,
    print_metrics,
    load_covertype_dataset,
)
from pytorch_tabular.utils import get_balanced_sampler, get_class_weighted_cross_entropy

# import os
%load_ext autoreload
%autoreload 2

In [2]:

Feature_Explain = pd.read_excel("data/Amex_ori/Amex Campus.xlsx")
Feature_target_cols = Feature_Explain["Feature Name"][
    Feature_Explain["Extended description"] == "Target"
].to_list()
Feature_cat_col_names = Feature_Explain["Feature Name"][
    Feature_Explain["Feature Type"] == "categorical"
].to_list()
Feature_num_col_names = Feature_Explain["Feature Name"][
    Feature_Explain["Variable Type"] == "numeric"
].to_list()

DEVICE = (
    "mps"
    if torch.backends.mps.is_available()
    else ("gpu" if torch.cuda.is_available() else "cpu")
)
print(DEVICE)

# %%
data = pd.read_parquet("data/Amex_ori/Amex Campus Challenge Train 3.parquet")

data=preprocess(data)

cols = data.columns.to_list()
target_cols = [col for col in cols if col in Feature_target_cols]
if len(target_cols) == 0:
    print("No target column found")
    exit()
# elif len(target_cols) > 1:
#     target_col=target_cols[1]
cat_col_names = [col for col in cols if col in Feature_cat_col_names]
num_col_names = [col for col in cols if col in Feature_num_col_names]

print(f"{target_cols}")
# target_col=cols.unite(Feature_target_cols)

# target_col = 'activation'

# %%

needed_cols = cat_col_names + num_col_names + target_cols
data = data[needed_cols]

data.head()
print(
    f"target_cols={target_cols}, cat_col_names={cat_col_names}, num_col_names={num_col_names}"
)

# %%
# Data: DF, cat_col_names, num_col_names

# 1. activate(0.57%) X recom(12.7%) -> 0~3

# _,data=train_test_split(data,test_size=0.05)



train, test = train_test_split(data)
# train, val = train_test_split(train,test_size=0.2)

print(f"train.shape={train.shape}, test.shape={test.shape}")


mps

['ind_recommended', 'activation']

target_cols=['ind_recommended', 'activation'], cat_col_names=['merchant_profile_01'], 
num_col_names=['customer_spend_06', 'customer_spend_07', 'customer_profile_01', 'customer_profile_02', 
'customer_profile_03', 'customer_digital_activity_02', 'customer_profile_04', 'distance_05']

train.shape=(29529, 11), test.shape=(9843, 11)

In [21]:
train[['activation', 'ind_recommended']].describe()


,activation,ind_recommended
count,29529.000000,29529.000000
mean,0.499001,0.497951
std,0.500007,0.500004
min,0.000000,0.000000
25%,0.000000,0.000000
50%,0.000000,0.000000
75%,1.000000,1.000000
max,1.000000,1.000000


In [3]:

test[['activation', 'ind_recommended']].describe()


,activation,ind_recommended
count,9843.000000,9843.000000
mean,0.493955,0.504826
std,0.499989,0.500002
min,0.000000,0.000000
25%,0.000000,0.000000
50%,0.000000,1.000000
75%,1.000000,1.000000
max,1.000000,1.000000


In [12]:

target_col = target_cols[0]

data_config = DataConfig(
    target=[target_col],
    # target should always be a list. Multi-targets are only supported for regression. Multi-Task Classification is not implemented
    continuous_cols=num_col_names,
    categorical_cols=cat_col_names,
    num_workers=0,
)
trainer_config = TrainerConfig(
    auto_lr_find=True,  # Runs the LRFinder to automatically derive a learning rate
    batch_size=1024,
    max_epochs=25,
    early_stopping=None,
    # early_stopping="valid_loss",  # Monitor valid_loss for early stopping
    # early_stopping_mode="min",  # Set the mode as min because for val_loss, lower is better
    # early_stopping_patience=5,  # No. of epochs of degradation training will wait before terminating
    checkpoints="valid_loss",  # Save best checkpoint monitoring val_loss
    checkpoints_path="checkpoints",  # Save the checkpoint in the experiment directory
    # checkpoints_save_top_k=5,
    # progress_bar="simple",
    load_best=True,  # After training, load the best checkpoint
    # accelerator=DEVICE,
    trainer_kwargs=dict(enable_model_summary=False),  # Turning off model summary
)
optimizer_config = OptimizerConfig()

head_config = LinearHeadConfig(
    layers="",  # No additional layer in head, just a mapping layer to output_dim
    dropout=0.1,
    initialization="kaiming",
).__dict__  # Convert to dict to pass to the model config (OmegaConf doesn't accept objects)



In [13]:
from pathlib import Path
def save_sweep_results(sweep_df, save_dir, best_model):
    if not Path(save_dir).exists:
        Path.mkdir(save_dir,parents=True)
    sweep_df.drop(columns=["params", "time_taken", "epochs"])
    sweep_df.to_excel(f"{save_dir}/sweep_result.xlsx", index=False)
    best_model.save_model(f"{save_dir}/best_model.ckpt")


def visualize_sweep_results(sweep_df):
    sweep_df.drop(columns=["params", "time_taken", "epochs"]).style.background_gradient(
        subset=["test_accuracy", "test_f1_score"], cmap="RdYlGn"
    ).background_gradient(subset=["time_taken_per_epoch", "test_loss"], cmap="RdYlGn_r")



In [14]:
%%time
# Filtering out the warnings
# t = time()

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    sweep_df, best_model = model_sweep(
        task="classification",  # One of "classification", "regression"
        train=train,
        test=test,
        data_config=data_config,
        optimizer_config=optimizer_config,
        trainer_config=trainer_config,
        model_list="lite",
        common_model_args=dict(head="LinearHead", head_config=head_config),
        metrics=["accuracy", "f1_score"],
        metrics_params=[{}, {"average": "macro"}],
        metrics_prob_input=[False, True],
        rank_metric=("accuracy", "higher_is_better"),
        progress_bar=True,
        verbose=True,
        suppress_lightning_logger=True,
    )
    



Output()

2024-03-07 02:19:38,684 - {pytorch_tabular:329} - INFO - Training CategoryEmbeddingModel

Finding best initial lr:   0%|          | 0/100 [00:00<?, ?it/s]

2024-03-07 02:19:53,598 - {pytorch_tabular:368} - INFO - Finished Training CategoryEmbeddingModel

2024-03-07 02:19:53,599 - {pytorch_tabular:369} - INFO - Results: model: CategoryEmbeddingModel, # Params: 52 T,   
epochs: 25, test_loss: 0.5194239616394043, test_accuracy: 0.7459814548492432, test_f1_score: 0.7455250024795532,   
time_taken: 14.926333904266357, time_taken_per_epoch: 0.5970533561706542

...

2024-03-07 02:21:36,759 - {pytorch_tabular:368} - INFO - Finished Training TabNetModel

2024-03-07 02:21:36,759 - {pytorch_tabular:369} - INFO - Results: model: TabNetModel, # Params: 51 T, epochs: 25,  
test_loss: 0.5258825421333313, test_accuracy: 0.7420194745063782, test_f1_score: 0.7412970662117004, time_taken:   
72.42718410491943, time_taken_per_epoch: 2.897087364196777

2024-03-07 02:21:36,761 - {pytorch_tabular:387} - INFO - Model Sweep Finished

2024-03-07 02:21:36,761 - {pytorch_tabular:388} - INFO - Best Model: CategoryEmbeddingModel

CPU times: user 1min 36s, sys: 7.75 s, total: 1min 43s
Wall time: 1min 58s


In [15]:

visualize_sweep_results(sweep_df)
save_sweep_results(sweep_df, "sweep_results",best_model)


2024-03-07 02:21:36,862 - {pytorch_tabular.tabular_model:1533} - WARNING - Directory is not empty. Overwriting the 
contents.